In [ ]:
%load_ext autoreload
%autoreload 2

In [69]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('data/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

# Full Implementation of Backprop

import numpy as np
# from data_prep import features, targets, features_test, targets_test

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape

last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

# for e in range(epochs):
#     del_w_input_hidden = np.zeros(weights_input_hidden.shape)
#     del_w_hidden_output = np.zeros(weights_hidden_output.shape)
#     for x, y in zip(features.values, targets):
#         # Forward pass #
#         hidden_input = np.dot(x, weights_input_hidden)
#         hidden_output = sigmoid(hidden_input)
        
#         final_input = np.dot(hidden_output, weights_hidden_output)
#         output = sigmoid(final_input)
        
#         error = y - output
        
#         #######
#         ### Output - Hidden unit ###
#         #######
        
#         # Error gradient in output unit
#         output_error = error * sigmoid_prime(final_input)
        
#         # Weight update in output-hidden units
#         del_w_hidden_output += output_error * hidden_output
        
#         #######
#         ### Hidden - Input unit ###
#         #######
        
#         # Error gradient in hidden units
#         hidden_error = output_error * weights_hidden_output * sigmoid_prime(hidden_input)
        
#         # Weight update in hidden-input units
#         del_w_input_hidden += np.matrix(x).T * np.matrix(hidden_error)
        
#     weights_input_hidden += learnrate * del_w_input_hidden / n_records
#     weights_hidden_output += learnrate * del_w_hidden_output / n_records

#     # Printing out the mean square error on the training set
#     if e % (epochs / 10) == 0:
#         hidden_output = sigmoid(np.dot(x, weights_input_hidden))
#         out = sigmoid(np.dot(hidden_output,
#                              weights_hidden_output))
#         loss = np.mean((out - targets) ** 2)

#         if last_loss and last_loss < loss:
#             print("Train loss: ", loss, "  WARNING - Loss Increasing")
#         else:
#             print("Train loss: ", loss)
#         last_loss = loss

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)

        output = sigmoid(np.dot(hidden_output,
                                weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate error gradient in output unit
        output_error = error * output * (1 - output)

        # TODO: propagate errors to hidden layer
        hidden_error = np.dot(output_error, weights_hidden_output) * \
                       hidden_output * (1 - hidden_output)

        # TODO: Update the change in weights
        del_w_hidden_output += output_error * hidden_output
        del_w_input_hidden += hidden_error * x[:, None]

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

/Users/amoghparam/workspace/amogh/sdc/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


('Train loss: ', 0.25135725242598811)
('Train loss: ', 0.24996540718842905)
('Train loss: ', 0.24862005218904504)
('Train loss: ', 0.2473199321717981)
('Train loss: ', 0.24606380465584854)
('Train loss: ', 0.24485044179257037)
('Train loss: ', 0.24367863201868301)
('Train loss: ', 0.24254718151769472)
('Train loss: ', 0.24145491550165454)
('Train loss: ', 0.24040067932493334)
Prediction accuracy: 0.725


In [84]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('data/binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

# Full Implementation of Backprop

import numpy as np
# from data_prep import features, targets, features_test, targets_test

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape

last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        # Forward pass #
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        
        final_input = np.dot(hidden_output, weights_hidden_output)
        output = sigmoid(final_input)
        
        error = y - output
        
        #######
        ### Output - Hidden unit ###
        #######
        
        # Error gradient in output unit
        output_error = error * sigmoid_prime(final_input)
        
        # Weight update in output-hidden units
        del_w_hidden_output += output_error * hidden_output
        
        #######
        ### Hidden - Input unit ###
        #######
        
        # Error gradient in hidden units
        hidden_error = np.dot(output_error, weights_hidden_output) * sigmoid_prime(hidden_input)
        
        # Weight update in hidden-input units
        del_w_input_hidden += hidden_error * x[:, None]
        
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

/Users/amoghparam/workspace/amogh/sdc/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


('Train loss: ', 0.25135725242598811)
('Train loss: ', 0.24996540718842905)
('Train loss: ', 0.24862005218904504)
('Train loss: ', 0.2473199321717981)
('Train loss: ', 0.24606380465584854)
('Train loss: ', 0.24485044179257037)
('Train loss: ', 0.24367863201868301)
('Train loss: ', 0.24254718151769472)
('Train loss: ', 0.24145491550165454)
('Train loss: ', 0.24040067932493334)
Prediction accuracy: 0.725


In [74]:
x

array([-1.27860482, -1.6551629 ,  0.        ,  1.        ,  0.        ,  0.        ])

In [75]:
hidden_error

array([-0.01328672,  0.01026319])

In [81]:
np.dot(x[:,None], hidden_error[:,None].T)

array([[ 0.01698847, -0.01312256],
       [ 0.02199169, -0.01698725],
       [ 0.        ,  0.        ],
       [-0.01328672,  0.01026319],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [73]:
np.matrix(x).T * np.matrix(hidden_error)

matrix([[ 0.01698847, -0.01312256],
        [ 0.02199169, -0.01698725],
        [ 0.        ,  0.        ],
        [-0.01328672,  0.01026319],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]])